[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/swarm-ai-safety/swarm/blob/main/examples/parameter_sweep.ipynb)

# Parameter Sweep: Governance Effects Study

This notebook demonstrates how to sweep governance parameters across a grid of configurations and compare their effects on ecosystem health. We vary the **transaction tax rate** and **circuit breaker** toggle to study how these governance levers influence welfare, toxicity, and agent payoffs.

**No API keys needed. Runs entirely locally (or in Colab).**

**Difficulty:** Intermediate

In [ ]:
# --- Setup ---
# This cell handles installation automatically.
# In Colab: clones the repo and installs SWARM.
# Locally: assumes you've already run `pip install -e ".[runtime]"`.
import os

if os.getenv("COLAB_RELEASE_TAG"):
    !git clone --depth 1 https://github.com/swarm-ai-safety/swarm.git /content/swarm
    %pip install -q -e "/content/swarm[runtime]"
    os.chdir("/content/swarm")
    print("Installed SWARM from GitHub. Ready to go!")
else:
    print("Local environment detected \u2014 using existing install.")

## What are we sweeping?

We sweep two governance parameters across all combinations:

| Parameter | Values | What it does |
|-----------|--------|--------------|
| **Transaction tax rate** | 0%, 5%, 10%, 15% | A per-interaction tax that funds governance infrastructure. Higher rates dampen exploitative behavior but also reduce net payoffs for honest agents. |
| **Circuit breaker** | Off / On | When enabled, the circuit breaker halts interactions if ecosystem toxicity spikes above a threshold, preventing cascading failures. |

With 4 tax rates x 2 circuit breaker states x 2 runs each = **16 total runs**. Each run uses a different random seed for statistical robustness.

In [ ]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

from swarm.analysis import SweepConfig, SweepParameter, SweepRunner
from swarm.scenarios import load_scenario

SCENARIOS_DIR = Path("scenarios") if Path("scenarios").is_dir() else Path("../scenarios")

In [ ]:
# Load the baseline scenario and reduce epochs for faster iteration
base_scenario = load_scenario(SCENARIOS_DIR / "baseline.yaml")
base_scenario.orchestrator_config.n_epochs = 5

# Configure the parameter sweep
sweep_config = SweepConfig(
    base_scenario=base_scenario,
    parameters=[
        SweepParameter(name="governance.transaction_tax_rate", values=[0.0, 0.05, 0.10, 0.15]),
        SweepParameter(name="governance.circuit_breaker_enabled", values=[False, True]),
    ],
    runs_per_config=2,
    seed_base=42,
)
print(f"Total runs: {sweep_config.total_runs()}")

def progress(current, total, params):
    param_str = ", ".join(f"{k}={v}" for k, v in params.items())
    print(f"  [{current}/{total}] {param_str}")

runner = SweepRunner(sweep_config, progress_callback=progress)
runner.run()
print("\nSweep complete!")

In [ ]:
# Extract results into a DataFrame
summary = runner.summary()
print(f"Total runs: {summary['total_runs']}")
print(f"Parameter combinations: {summary['param_combinations']}")
print()

# Build a summary DataFrame
rows = []
for s in summary["summaries"]:
    rows.append({
        "tax_rate": s.get("governance.transaction_tax_rate", 0.0),
        "circuit_breaker": s.get("governance.circuit_breaker_enabled", False),
        "mean_welfare": s["mean_welfare"],
        "mean_toxicity": s["mean_toxicity"],
        "mean_frozen": s["mean_frozen"],
        "mean_honest_payoff": s["mean_honest_payoff"],
        "mean_adversarial_payoff": s["mean_adversarial_payoff"],
    })

df_summary = pd.DataFrame(rows)
df_summary["circuit_breaker"] = df_summary["circuit_breaker"].map({True: "On", False: "Off"})
df_summary

## Results

The table above shows the mean welfare, toxicity, frozen agent count, and per-type payoffs for each parameter combination (averaged across 2 runs each).

Key columns:
- **mean_welfare**: Total ecosystem welfare summed across all epochs. Higher is better.
- **mean_toxicity**: Average toxicity rate `E[1-p | accepted]`. Lower is better.
- **mean_frozen**: Average number of agents frozen by governance. More frozen agents means governance is actively intervening.
- **mean_honest_payoff / mean_adversarial_payoff**: How much honest vs adversarial agents earn on average. Ideally, honest agents earn more.

In [ ]:
# Visualization: welfare and toxicity across tax rates, grouped by circuit breaker
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle("Governance Effects: Tax Rate vs Circuit Breaker", fontsize=14)

cb_off = df_summary[df_summary["circuit_breaker"] == "Off"]
cb_on = df_summary[df_summary["circuit_breaker"] == "On"]

# --- Welfare ---
ax = axes[0]
x = cb_off["tax_rate"]
width = 0.015
ax.bar(x - width, cb_off["mean_welfare"].values, width=width * 2, label="CB Off", color="#1f77b4", alpha=0.8)
ax.bar(x + width, cb_on["mean_welfare"].values, width=width * 2, label="CB On", color="#2ca02c", alpha=0.8)
ax.set_xlabel("Transaction Tax Rate")
ax.set_ylabel("Mean Welfare")
ax.set_title("Welfare")
ax.legend()
ax.set_xticks([0.0, 0.05, 0.10, 0.15])

# --- Toxicity ---
ax = axes[1]
ax.bar(x - width, cb_off["mean_toxicity"].values, width=width * 2, label="CB Off", color="#1f77b4", alpha=0.8)
ax.bar(x + width, cb_on["mean_toxicity"].values, width=width * 2, label="CB On", color="#2ca02c", alpha=0.8)
ax.set_xlabel("Transaction Tax Rate")
ax.set_ylabel("Mean Toxicity")
ax.set_title("Toxicity")
ax.legend()
ax.set_xticks([0.0, 0.05, 0.10, 0.15])

# --- Honest vs Adversarial Payoff ---
ax = axes[2]
ax.plot(cb_off["tax_rate"].values, cb_off["mean_honest_payoff"].values, "o-", label="Honest (CB Off)", color="#2ca02c")
ax.plot(cb_on["tax_rate"].values, cb_on["mean_honest_payoff"].values, "s--", label="Honest (CB On)", color="#2ca02c", alpha=0.6)
ax.plot(cb_off["tax_rate"].values, cb_off["mean_adversarial_payoff"].values, "o-", label="Adversarial (CB Off)", color="#d62728")
ax.plot(cb_on["tax_rate"].values, cb_on["mean_adversarial_payoff"].values, "s--", label="Adversarial (CB On)", color="#d62728", alpha=0.6)
ax.set_xlabel("Transaction Tax Rate")
ax.set_ylabel("Mean Payoff")
ax.set_title("Agent Payoffs by Type")
ax.legend(fontsize=8)
ax.set_xticks([0.0, 0.05, 0.10, 0.15])

plt.tight_layout()
plt.show()

## Key Findings and Next Steps

**Observations from the sweep:**

- **Transaction tax** acts as a blunt instrument: it reduces adversarial payoffs but also dampens honest agent earnings. There is typically a sweet spot (around 5-10%) where the tradeoff is favorable.
- **Circuit breakers** provide a safety floor by halting interactions during toxicity spikes. When combined with moderate tax rates, they tend to improve welfare without excessive honest-agent cost.
- The interaction between the two parameters is non-trivial: circuit breakers may matter less at high tax rates (where exploitation is already suppressed) and more at low tax rates (where they catch runaway toxicity).

**Next steps:**

- Sweep additional parameters (e.g., `governance.audit_probability`, `governance.staking_amount`) using the same `SweepConfig` API.
- Increase `runs_per_config` to 5+ for tighter confidence intervals.
- Use `runner.to_csv("results.csv")` to export raw per-run data for offline analysis.
- Try the adversarial red-team scenario as the base to study governance under stress.

**See also:** `examples/quickstart.ipynb` for an introduction to SWARM, or `examples/parameter_sweep.py` for the CLI version of this sweep.